In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
import tensorflow as tf

In [ ]:
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import layers

In [ ]:
from PIL import Image
import glob
import os
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import ImageDataGenerator
from sklearn.model_selection import train_test_split

import numpy as np

In [ ]:
def load_images_in_folder(folder):
  images = []
  for filename in os.listdir(folder):
    img = image.load_img(os.path.join(folder, filename), target_size=(224, 224))
    if img is not None:
      x = image.img_to_array(img)
      x = x / 255
      images.append(x)
  return images

corona_train = load_images_in_folder('./')
normal_train = load_images_in_folder('./')
pneu_train = load_images_in_folder('./')

corona_test = load_images_in_folder('./')
normal_test = load_images_in_folder('./')
pneu_test = load_images_in_folder('./')

In [ ]:
corona_train = np.array(corona_train)
corona_test = np.array(corona_test)

normal_train = np.array(normal_train)
normal_test = np.array(normal_test)

pneu_train = np.array(pneu_train)
pneu_test = np.array(pneu_test)

In [ ]:
train_gen = ImageDataGenerator(rescale = 1./255,
                               shear_range = 0.2,
                               width_shift_range = 0.1,
                               height_shift_range = 0.1,
                               horizontal_flip = True,
                               fill_mode = 'nearest')
test_gen = ImageDataGenerator(rescale = 1. / 255,)

In [ ]:
train_dir = './'
test_dir = './'
IMAGE_SHAPE = 224
batch_size = 32

train_data_gen = train_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMAGE_SHAPE, IMAGE_SHAPE),
                                               class_mode='sparse'
                                               )
test_data_gen = test_gen.flow_from_directory(directory=test_dir,
                                             target_size = (IMAGE_SHAPE, IMAGE_SHAPE),
                                             batch_size=batch_size,
                                             class_mode='sparse'
                                             )

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('corona_model_callback10.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          verbose=1,
                          restore_best_weights=True,
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1, 
                              patience=4, 
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop, checkpoint, reduce_lr]

In [ ]:
def plotImages(images_arr):
  fig, axes = plt.subplots(1, 5, figsize=(20, 20))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
  plt.tight_layout()
  plt.show()

augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

model = Sequential()
model.add(Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMAGE_SHAPE, IMAGE_SHAPE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(3))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

EPOCHS = 25

history = model.fit_generator(train_data_gen, 
                              steps_per_epoch = int(np.ceil(train_data_gen.n/ float(batch_size))),
                              epochs=EPOCHS,
                              callbacks=callbacks,
                              validation_data=test_data_gen,
                              validation_steps=int(ceil(test_data_gen.n/ float(batch_size)))
                              )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(7)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training loss')
plt.plot(epochs_range, val_loss, label='Validation loss')
plt.legend(loc='upper right')
plt.title('Training and Validation loss')
plt.show()

In [ ]:
result = model.predict_classes(corona_test[0][np.newaxis, ...])
result

In [ ]:
model.predict_classes(normal_test[19][np.newaxis, ...])

In [ ]:
model.save('dirty_covid_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

classifier = load_model('dirty_covid_model.h5')